# Домашнее задание

Способы выделения продуктов категории CDs&Vinyl:

1. Rule-based модель spacy (реализована). Из метаинформации понадобятся названия продуктов. Также я брала токены cd, album, disc, vinyl, record, LP и их сочетания со словами, начинающимися с заглавных букв.

2. Составить словарь сущностей (можно взять из прошлого пункта). Потом расширить его с помощью эмбеддингов, посмотрев ближайшие к ним. 

3. Можно попробовать сделать классификатор, который будет предсказывать категорию продуктов, и модель, которая будет предсказывать названия конкретных продуктов. Потом взять слова, у которых будут наибольшие веса, первая модель должна выдать что-то в духе cd, album и т.д., вторая - названия (в отличие от 1 пункта, здесь легче должны считываться неполные названия конкретных продуктов)

4. Вариант трудоемкий: сначала разметить часть данных вручную/через ruled-based модель, потом обучить на этих данных модель из allenlp, например. 

### Реализация 1 варианта:

In [2]:
import pandas as pd
import gzip
from tqdm import tqdm
import spacy
from spacy.matcher import PhraseMatcher, Matcher
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist, bigrams
from nltk.collocations import *
from nltk.metrics.spearman import *

Создание датафреймов с отзывами и метаинформацией.

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_CDs_and_Vinyl_5.json.gz')
meta = getDF('meta_CDs_and_Vinyl.json.gz')

Создание паттернов, извлечение наваний продуктов и биграм с ними (брала только первые 100000 отзывов, про многих нет метаинформации, поэтому всего получился корпус из 46745 отзывов).

Паттерны:

1) Названия продуктов

2) cd/album/vinyl/disc/LP/record + (опционально) ('/") Слова_с_заглавной_буквы ('/")

In [4]:
patterns = [[{'LOWER': 'cd'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'album'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'disc'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'vinyl'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'record'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'TEXT': 'LP'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}]]
terms = []
rws = []
for id_p in tqdm(set(df['asin'][:100000]) & set(meta['asin'])):
    tt = meta.loc[meta['asin'] == id_p]['title'].tolist()[0]
    rw = df.loc[df['asin'] == id_p]['reviewText'].tolist()
    rws.extend(rw)
    terms.append(tt)

100%|██████████| 3028/3028 [09:56<00:00,  5.08it/s]


In [5]:
nlp = spacy.load("en_core_web_sm")
ph_matcher = PhraseMatcher(nlp.vocab)
matcher = Matcher(nlp.vocab)
ph_patterns = [nlp.make_doc(text) for text in tqdm(terms)]
ph_matcher.add('PRD', ph_patterns)
matcher.add('PRD', patterns)
docs = [nlp.make_doc(text) for text in tqdm(rws)]
matches = [matcher(doc) for doc in tqdm(docs)]
ph_matches = [ph_matcher(doc) for doc in tqdm(docs)]

100%|██████████| 46745/46745 [00:16<00:00, 2770.04it/s] 


In [6]:
ngrams = []
results = []
for match, ph_match, doc in zip(matches, ph_matches, docs):
    res = []
    for match_id, start, end in match:
        span = doc[start:end]
        res.append(span.text)
        if start > 0:
            ngrams.append((doc[start-1].text, span.text))
        if end < len(doc) - 1:
            ngrams.append((span.text, doc[end+1].text))
    for match_id, start, end in ph_match:
        span = doc[start:end]
        res.append(span.text)
        if start > 0:
            ngrams.append((doc[start-1].text, span.text))
        if end < len(doc) - 1:
            ngrams.append((span.text, doc[end+1].text))
    results.append(res)

In [7]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
words = []
ngs = []
for rw, r, ng in tqdm(zip(rws, results, ngrams)):
    ws = word_tokenize(rw)
    words.extend(ws)
    words.extend([w for w in r if w not in ws])
    ngs.extend(list(bigrams(ws)))
    ngs.extend([n for n in ng if n not in ngs])
w_fd = FreqDist(words)
bg_fd = FreqDist(ngs)

46745it [22:48, 34.16it/s] 


In [ ]:
finder = BigramCollocationFinder(w_fd, bg_fd)
finder.apply_freq_filter(3)
nes = []
for r in results:
    nes.extend(r)
nes = set(nes)
sw = nltk.corpus.stopwords.words('english')
finder.apply_ngram_filter(lambda w1, w2: (w1 not in nes and w2 not in nes) or w1 in sw or w2 in sw or w1.isalpha() == False or w2.isalpha() == False)

In [ ]:
sc_pmi = finder.score_ngrams(bigram_measures.pmi)
sc_t = finder.score_ngrams(bigram_measures.student_t)
sc_ll = finder.score_ngrams(bigram_measures.likelihood_ratio)

In [ ]:
list(ranks_from_scores(sc_pmi))

In [ ]:
list(ranks_from_scores(sc_t))

In [ ]:
list(ranks_from_scores(sc_ll))

In [ ]:
ne_groups = {}
for n, r in list(ranks_from_scores(sc_t)):
    for w in n:
        if w in nes:
            if w not in ne_groups:
                ne_groups[w] = [(n[0] + ' ' + n[1], r)]
            else:
                ne_groups[w].append((n[0] + ' ' + n[1], r))

In [ ]:
ne_groups['album'][:5]

In [ ]:
ne_groups['cd'][:5]

In [ ]:
ne_groups['disc'][:5]

In [ ]:
ne_groups['vinyl'][:5]

In [61]:
ne_groups['set'][:5]

KeyError: 'dvd'

Борьба с синонимами:

1) Можно попробовать воспользоваться синсетами из wordnet или его аналогов

2) Также часто можно объединять однокоренные слова/ н-грамм с общей вершиной